In [ ]:
from nbdev import *
%nbdev_default_export control.nodes

Cells will be exported to pct.control.nodes,
unless a different module is specified after an export flag: `%nbdev_export special.module`
